## <center> Project-4. Решение комплексной бизнес-задачи

### Цели
Подготовить основу рекомендательной системы, благодаря которой можно будет предлагать клиентам интересные им курсы и тем самым повышать средний чек.

### Формализованные задачи
1. Познакомиться с датасетом, подготовить и проанализировать данные с помощью SQL.
2. Обработать данные средствами Python.
3. Составить итоговую таблицу с рекомендациями.

### Принятые в анализе предпосылки
Таблица рекомендаций приведена для 126 курсов, т.к. один курс, согласно имеющимся данным, никто не покупал (всего различных курсов 127).

### Анализ данных
#### 1. *Подготовим датасет с помощью SQL с данными по продажам курсов в разрезе пользователей (купивших более одного курса).*


SQL-запрос:  
with all_user AS 
	(SELECT user_id,  
		 resource_id  
	FROM final.carts c  
	JOIN final.cart_items ci  
		ON c.id=ci.cart_id  
	WHERE state='successful'  
			AND resource_type='Course'),  
count_res AS  
	(SELECT user_id,  
		 count(distinct resource_id) cnt  
	FROM all_user  
	GROUP BY  1  
	HAVING count(distinct resource_id) > 1)  
 SELECT DISTINCT cr.user_id,  
		resource_id  
 FROM count_res cr  
 JOIN all_user au  
	ON cr.user_id=au.user_id  
##### В результате запроса мы получили датасет, который содержит 34074 строк.


#### 2. *Обработаем данные средствами Python*

In [1]:
# Импортируем необходимые библиотеки
import pandas as pd
import itertools as it
from collections import Counter
import numpy as np
import psycopg2
import psycopg2.extras

In [2]:
# Выполним функцию запроса к БД с помощью вышеприведенного SQL запроса, а затем запишем полученные данные в датафрейм
def getUsersCount():
    query = '''with all_user AS 
	(SELECT user_id,
		 resource_id
	FROM final.carts c
	JOIN final.cart_items ci
		ON c.id=ci.cart_id
	WHERE state='successful'
			AND resource_type='Course'), 
count_res AS 
	(SELECT user_id,
		 count(distinct resource_id) cnt
	FROM all_user
	GROUP BY  1
	HAVING count(distinct resource_id) > 1)
SELECT DISTINCT cr.user_id,
		resource_id
FROM count_res cr
JOIN all_user au
	ON cr.user_id=au.user_id
    '''.format()
    conn = psycopg2.connect("dbname='skillfactory' user='skillfactory' host='84.201.134.129' password='cCkxxLVrDE8EbvjueeMedPKt' port=5432")
    dict_cur = conn.cursor(cursor_factory=psycopg2.extras.DictCursor)
    dict_cur.execute(query)
    rows = dict_cur.fetchall()
    data = []
    for row in rows:
        data.append(dict(row))
    return data
data = pd.DataFrame(getUsersCount())
# Выведем общую информацию о датафрейме. Пропущенных значений нет, тип столбцов - целочисленные, всего строк 34074.
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34074 entries, 0 to 34073
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype
---  ------       --------------  -----
 0   user_id      34074 non-null  int64
 1   resource_id  34074 non-null  int64
dtypes: int64(2)
memory usage: 532.5 KB


In [3]:
# Сформируем для каждого пользователя список купленных им курсов (уникальные id курсов, отсортированные по возрастанию номера)
df = data.groupby(['user_id'])['resource_id'].apply(lambda x: list(np.unique(x))).reset_index()
df.head()

,user_id,resource_id
0,51,"[516, 1099]"
1,6117,"[356, 357, 1125]"
2,10275,"[553, 1147]"
3,10457,"[361, 1138]"
4,17166,"[356, 357]"


In [4]:
# Применим к полученным спискам id курсов функцию, которая формирует список пар возможных комбинаций покупок курсов для каждого пользователя
df_list = df['resource_id'].apply(lambda x: list(it.combinations(x,2)))
df_list.head()

0                             [(516, 1099)]
1    [(356, 357), (356, 1125), (357, 1125)]
2                             [(553, 1147)]
3                             [(361, 1138)]
4                              [(356, 357)]
Name: resource_id, dtype: object

In [5]:
# Определим повторяемость каждой пары купленных курсов с помощью цикла
courses = []
for course in df_list:
    for el in course:
        courses.append(el)
count_courses = Counter(courses)
# Выведем какое количество различных пар курсов присутствует в данных
print(len(count_courses))
# Отобразим результат: повторяемость каждой пары курсов в порядке убывания. 
# Отметим, что наиболее популярная пара курсов с id 551-566.
count_courses

3989


Counter({(551, 566): 797,
         (515, 551): 417,
         (489, 551): 311,
         (523, 551): 304,
         (566, 794): 290,
         (489, 515): 286,
         (490, 566): 253,
         (490, 551): 247,
         (570, 752): 247,
         (569, 572): 216,
         (515, 523): 213,
         (553, 745): 212,
         (489, 523): 206,
         (569, 840): 204,
         (514, 551): 200,
         (516, 745): 199,
         (515, 566): 195,
         (489, 566): 188,
         (504, 572): 184,
         (572, 840): 178,
         (551, 552): 177,
         (507, 570): 172,
         (490, 809): 163,
         (489, 490): 152,
         (507, 752): 150,
         (523, 552): 144,
         (490, 515): 143,
         (551, 570): 142,
         (504, 569): 139,
         (514, 515): 139,
         (551, 745): 138,
         (514, 566): 138,
         (502, 551): 135,
         (504, 840): 135,
         (571, 1125): 122,
         (502, 566): 120,
         (523, 566): 120,
         (570, 809): 119,
         (7

#### 3. *Составим таблицу рекомендаций для каждого курса*

In [6]:
# Определим минимальную границу - какое количество раз считаем слишком малым для вывода курса в рекомендации
def getcountcourses(*counter_data):
    count_list = []
    for cnt in counter_data:
        count_list.append(cnt)
    return count_list
# Возьмем из count_courses только значения количества повторений
min_rec=getcountcourses(*count_courses.values())
min_rec_df=pd.DataFrame(min_rec)
# Оценим общую информацию по полученным данным
min_rec_df.describe()

,0
count,3989.000000
mean,10.031838
std,26.355998
min,1.000000
25%,1.000000
50%,3.000000
75%,9.000000
max,797.000000


Примем за минимальную границу учета курса в рекомендациях - значение 3 квартиля (75%) - 9 раз.  
Это значение примерно совпадает со средним значением частоты покупок.  
Наблюдается большой разрыв между частотностью покупок по всем пара:  
75% покупок пар совершается менее 9 раз, при этом частотность у топ-30 пар более 100 раз.  
Т.к. наша задача увеличить продажи курсов, то не будем учитывать наименее популярные пары курсов при составлении рекомендаций.

In [7]:
# Зададим значение минимальной границы учета курсов в рекомендациях
min_count = 9
# Сформируем список уникальных id курсов, для которых будем формировать рекомендательную таблицу.
set_course = data['resource_id'].unique()

In [8]:
# Создадим функцию, которая будет выводить для каждого уникального id курса две пары наиболее популярного сочетания курсов 
# с указанием частотности покупок.
def reccom(course_id):
    list_pairs = []
    for i in count_courses.keys():
        if i[0] == course_id:
            list_pairs.append((i,count_courses[i]))
        if i[1] == course_id:
                list_pairs.append((i,count_courses[i]))
        list_pairs = sorted(list_pairs, reverse=True, key=lambda x: x[1])
    return list_pairs[0],list_pairs[1]

In [9]:
# Заполним список рекомендаций с id курса и двумя рекомендованными курсами с учетом частотности покупок.
# Если частотность покупок меньше установленной нами границы (9), то выводим в рекомендации наиболее популярные курсы 551,566.
list_recom = []
for el in set_course:
    if reccom(el)[0][0][0] == el and reccom(el)[0][1] >= min_count:
        rec_1 = reccom(el)[0][0][1]
    elif reccom(el)[0][0][0] != el and reccom(el)[0][1] >= min_count:
           rec_1 = reccom(el)[0][0][0]
    if reccom(el)[1][0][0] == el and reccom(el)[1][1] >= min_count:
            rec_2 = reccom(el)[1][0][1]
    elif reccom(el)[1][0][0] != el and reccom(el)[1][1] >= min_count:
            rec_2 = reccom(el)[1][0][0]
    else:
        rec_1 = 551
        rec_2 = 566
    list_recom.append((el, rec_1, rec_2))
# Создадим из полученного списка датафрейм рекомендаций
df_recomm = pd.DataFrame(list_recom,columns=['course_id','recomm_1','recomm_2'])
# Отсортируем таблицу по возрастанию id курсов
df_recomm = df_recomm.sort_values(by=['course_id'], ignore_index=True)

#### Таблица рекомендаций
В полученной таблице рекомендаций приведены:  
в столбце "сourse_id" - уникальные id курсов, для которых приведена рекомендация;  
в столбце "recomm_1" - самый популярный курс для рекомендаций;  
в столбце "recomm_2" - второй по популярности курс.

In [10]:
df_recomm 

,course_id,recomm_1,recomm_2
0,356,571,357
1,357,571,356
2,358,570,752
3,359,570,358
4,360,745,516
...,...,...,...
121,1188,1141,513
122,1198,551,566
123,1199,551,566
124,1200,551,566


### Итоги
1. Согласно проведенному анализу данных, была составлена таблица рекомендаций для каждого id курса.  
2. В результате таблица содержит 126 курсов, которые были куплены (один курс не учитывался, т.к. в данных о покупках он отсутствует). 
3. Также было учтено минимальное количество покупок пары курсов и если частота покупок не превышала установленной нами границы (9 раз), то рекомендовались курсы из наиболее популярной пары.  
Мы предполагаем, что самые популярные курсы значительно больше покупают чем остальные, т.к. они грамотно составлены, содержат полезную для большинства пользоваталей информацию и могут быть интересны другим пользователям. И если покупателю понравится рекомендательный курс, то в дальнейшем он также может воспользоваться системой рекомендаций и докупить другие курсы.
4. При более глубоком анализе можно было бы учесть при формировании пар рекомендаций не только минимальную границу частотности покупки пары курсов, но и частотность покупки рекомендательного курса в других парах. Т.к. рекомендательный курс в данной паре мог быть куплен 1 раз, но в других сочетаниях пар мог быть куплен значительно больше раз, т.е. сам по себе он не является малопопулярным и вполне подходит для рекомендаций.  
Также следует проанализировать малопопулярные курсы на предмет содержащейся в ней информации, может быть они являются узкопрофильными и их как раз стоит профильно рекомендовать к другим таким же специфическим курсам.
